<a href="https://colab.research.google.com/github/sunyux/Pollution-the-Fashion-data-and-test-NN/blob/main/Pollution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch  
from torch import nn, optim , tensor
import torch.nn.functional as F 
from torchvision import datasets, transforms 
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset
import struct

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,), (0.5,))])
# datasets
trainset = datasets.FashionMNIST('./data',
 download=True,
 train=True,
 transform=transform)

testset =datasets.FashionMNIST('./data',
 download=True,
 train=False,
 transform=transform)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [ ]:
#load data
trainloader = torch.utils.data.DataLoader(trainset, batch_size=10,
 shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=10,
 shuffle=False, num_workers=2)


In [ ]:
#shift test image’s right by 2 pixels

class CustomDataset(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets


    def __getitem__(self, index):
      #change left and right
        x = torch.roll(self.data[index],shifts=2,dims=1)
        y = self.targets[index]
  
        return x, y

    def __len__(self):
        return len(self.data)

In [ ]:
#shift test image’s right by 2 pixels and down by 2 pixels

class CustomDataset1(Dataset):
    def __init__(self, data, targets):
        self.data = data
        self.targets = targets


    def __getitem__(self, index):
      #change top to the down
        x = torch.roll(self.data[index],shifts=(2,2),dims=(1,0))
        y = self.targets[index]
  
        return x, y

    def __len__(self):
        return len(self.data)

In [ ]:
#get shift data
from torch.utils.data import DataLoader
mydata = CustomDataset(testset.test_data, testset.test_labels)
test_dataloader = DataLoader(mydata, batch_size=10,shuffle=True, num_workers=0)
mydata1 = CustomDataset1(testset.test_data, testset.test_labels)
test_dataloader1 = DataLoader(mydata1, batch_size=10,shuffle=True, num_workers=0)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:57: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [ ]:
#pollution the data 

def load_mnist(path, kind='train'):
    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte'
                               % kind)
    with open(labels_path, 'rb') as lbpath:
        magic, n = struct.unpack('>II',
                                 lbpath.read(8))
        labels = np.fromfile(lbpath,
                             dtype=np.uint8)

    with open(images_path, 'rb') as imgpath:
        magic, num, rows, cols = struct.unpack('>IIII',
                                               imgpath.read(16))
        images = np.fromfile(imgpath,
                             dtype=np.uint8).reshape(len(labels), 784)

    return images, labels

X_train, y_train = load_mnist(r"./data/FashionMNIST/raw",kind='train')
X_test, y_test = load_mnist(r'./data/FashionMNIST/raw',kind='t10k')


import matplotlib.pyplot as plt

flag = 0
list = [0 for i in range(60000)]

for k in range (10):
  for i in range(1,10):
    for j in range (60000):
      if y_train[j] == k:
        if list[j] == 0:
          y_train[j] = (i + y_train[j] ) % 10
          list[j] = 1
          flag += 1
      if flag == 60:
        flag = 0
        break


In [ ]:
#got the pollution data
mydata = CustomDataset(X_train, y_train)
train_dataloader = DataLoader(mydata, batch_size=10, shuffle=True, num_workers=0)

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define model
class NeuralNetwork(nn.Module):
      def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(784, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 10)
        )

      def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits



In [ ]:
#set the criterion and optimizer
model = NeuralNetwork().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0)

In [ ]:
#train the model
def train(dataloader, model, criterion, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = criterion(pred, y.float())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
         #   print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
#test the model
def test(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += criterion(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {(100*correct):>0.1f}%")

In [ ]:
#test the model (for the shift)
def test2(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred=model(X.float())
            test_loss += criterion(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Accuracy: {(100*correct):>0.1f}%")

In [ ]:
epochs = 30
for t in range(epochs):
    print(f"Epoch  {t+1}\n-------------------------------")
    train(trainloader, model, criterion, optimizer)
    test(testloader, model, criterion)
    print(f"Epoch after pollution {t+1}\n-------------------------------")
    train(train_dataloader, model, criterion, optimizer)
    test(testloader, model, criterion)
print("Done!")

Epoch  1
-------------------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([10])) that is different to the input size (torch.Size([10, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Accuracy: 10.0%
Epoch after pollution 1
-------------------------------


TypeError: ignored

In [ ]:
#test!!!!
epochs = 30
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, criterion, optimizer)
    test(testloader, model, criterion)
    print(f"After shift left and right is \n-------------------------------")
    test2(test_dataloader, model, criterion)
    print(f"After shift top and down is \n-------------------------------")
    test2(test_dataloader1, model, criterion)
print("Done!")

Epoch 1
-------------------------------
Accuracy: 84.2%
After shift left and right is 
-------------------------------
Accuracy: 38.6%
After shift top and down is 
-------------------------------
Accuracy: 29.7%
Epoch 2
-------------------------------
Accuracy: 85.7%
After shift left and right is 
-------------------------------
Accuracy: 35.7%
After shift top and down is 
-------------------------------
Accuracy: 26.2%
Epoch 3
-------------------------------
Accuracy: 86.7%
After shift left and right is 
-------------------------------
Accuracy: 38.3%
After shift top and down is 
-------------------------------
Accuracy: 27.4%
Epoch 4
-------------------------------
Accuracy: 87.2%
After shift left and right is 
-------------------------------
Accuracy: 37.9%
After shift top and down is 
-------------------------------
Accuracy: 30.8%
Epoch 5
-------------------------------
Accuracy: 87.2%
After shift left and right is 
-------------------------------
Accuracy: 26.8%
After shift top a